In [1]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [3]:
df = pd.read_csv('C:/ds_work/ML_DATA/smoking.csv')
df.shape  # (55693, 27)

(55692, 27)

In [4]:
# 범주형 데이터 변환
df = df.replace({'N':0,'Y':1,'F':0,'M':1, })
gender = {'F': 0, 'M': 1}
oral = {'N': 0, 'T': 1}
tartar = {'N': 0, 'T': 1}
# df 전체 float형으로 바꾸기
df = df.astype(float)

In [5]:
target = df.loc[:, 'smoking']
#target
df_li = df.iloc[:, 10:18]
#df_li
df_bl = df.iloc[:, 20:23]
#df_bl
li_bl = pd.concat([df_li,df_bl],axis=1)
#li_bl
df = pd.concat([li_bl,target],axis=1)
df

,systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,AST,ALT,Gtp,smoking
0,114.0,73.0,94.0,215.0,82.0,73.0,126.0,12.9,18.0,19.0,27.0,0.0
1,119.0,70.0,130.0,192.0,115.0,42.0,127.0,12.7,22.0,19.0,18.0,0.0
2,138.0,86.0,89.0,242.0,182.0,55.0,151.0,15.8,21.0,16.0,22.0,1.0
3,100.0,60.0,96.0,322.0,254.0,45.0,226.0,14.7,19.0,26.0,18.0,0.0
4,120.0,74.0,80.0,184.0,74.0,62.0,107.0,12.5,16.0,14.0,22.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
55687,110.0,68.0,89.0,213.0,99.0,75.0,118.0,12.3,14.0,7.0,10.0,0.0
55688,101.0,62.0,89.0,166.0,69.0,73.0,79.0,14.0,20.0,12.0,14.0,0.0
55689,117.0,72.0,88.0,158.0,77.0,79.0,63.0,12.4,17.0,11.0,12.0,0.0
55690,133.0,76.0,107.0,210.0,79.0,48.0,146.0,14.4,20.0,19.0,18.0,0.0


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
train_test_split(df.iloc[:,:-1], df.iloc[:,-1], test_size=0.2, random_state=42)

In [7]:
#스케일링
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()

X_train = stdsc.fit_transform(X_train)
X_test = stdsc.transform(X_test)

In [9]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import pandas as pd
import xgboost as xgb
import pandas as pd
import seaborn as sns

estimator = xgb.XGBClassifier(early_stopping_rounds=10)
eval_set = [(X_test, y_test)]
estimator.fit(X_train, y_train, eval_set=eval_set, verbose=False)

pred = estimator.predict(X_test)
acc = accuracy_score(y_test,pred)
acc

criterion_list =  ['gini','entropy'] 
splitter_list =  ['best','random']
max_depth_list =  np.arange(2, 20)
#min_samples_split_list = np.arange(2, 20)                                   # 노드 분할 위한 최소 표본수 작으면 과적합, 반대의 경우 과소적합
min_weight_fraction_leaf_list =  np.append(0.0,np.arange(0.1, 0.5, 0.1))    # 비율 커지면 과대적합 줄어듬 0,5보다 작게 해야한다
#min_impurity_decrease_list = np.append(0.0,np.arange(0.3, 0.5, 0.1))        # 비율 노드분할 관련

parameter={
    'criterion' : criterion_list ,
    'splitter'  : splitter_list,
    'max_depth' : max_depth_list,
    'min_weight_fraction_leaf' : min_weight_fraction_leaf_list,    
    'random_state': [42]
}

model = GridSearchCV(estimator, param_grid=parameter, cv=3, verbose=1, n_jobs=-1, refit=True)
# verbose 출력 실행 모듈
model.fit(X_train,y_train)

print('best_params : ', model.best_params_)
print('best_score_ : ', model.best_score_)

gc.paramsTocsv(data_name='liverblood', scaler='StandardScaler', model_name='Xgboost', best_params=model.best_params_, best_score=model.best_score_)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=10, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)

0.73893527246611

Fitting 3 folds for each of 360 candidates, totalling 1080 fits


C:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
1080 fits failed out of a total of 1080.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1080 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\msi\AppData\Roaming\Python\Python39\site-packages\xgboost\core.py", line 620, in inner_f
    return func(**kwargs)
  File "C:\Users\msi\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py", line 1516, in fit
    self._Booster = train(
  File "C:\Users\msi\AppData\Roaming\Python\Python39\site-packa

[15:04:04] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "criterion", "min_weight_fraction_leaf", "splitter" } are not used.



AssertionError: Must have at least 1 validation dataset for early stopping.